In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg,max

spark = SparkSession.builder \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

In [0]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NV",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","DE",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","NV",80000,25,18000),
    ("Kumar","Marketing","NJ",91000,50,21000)
  ]

In [0]:
schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NV   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |DE   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |NV   |80000 |25 |18000|
|Kumar        |Marketing |NJ   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [0]:
df.groupBy("state").sum("salary").show()

+-----+-----------+
|state|sum(salary)|
+-----+-----------+
|   NY|     252000|
|   NV|     166000|
|   CA|     171000|
|   DE|      99000|
|   NJ|      91000|
+-----+-----------+



In [0]:
dfGroup=df.groupBy("state") \
          .agg(sum("salary").alias("sum_salary"))

In [0]:
dfGroup.show(truncate=False)

+-----+----------+
|state|sum_salary|
+-----+----------+
|NY   |252000    |
|NV   |166000    |
|CA   |171000    |
|DE   |99000     |
|NJ   |91000     |
+-----+----------+



In [0]:
dfFilter=dfGroup.filter(dfGroup.sum_salary > 100000)
dfFilter.show()


+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   NV|    166000|
|   CA|    171000|
+-----+----------+



In [0]:
from pyspark.sql.functions import asc
dfFilter.sort("sum_salary").show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NV|    166000|
|   CA|    171000|
|   NY|    252000|
+-----+----------+



In [0]:
from pyspark.sql.functions import desc
dfFilter.sort(desc("sum_salary")).show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   CA|    171000|
|   NV|    166000|
+-----+----------+



In [0]:
df.groupBy("state") \
  .agg(sum("salary").alias("sum_salary")) \
  .filter(col("sum_salary") > 100000)  \
  .sort(desc("sum_salary")) \
  .show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   CA|    171000|
|   NV|    166000|
+-----+----------+



In [0]:
df.createOrReplaceTempView("EMP")
spark.sql("select state, sum(salary) as sum_salary from EMP " +
          "group by state having sum_salary > 100000 " + 
          "order by sum_salary desc").show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   CA|    171000|
|   NV|    166000|
+-----+----------+



In [0]:
df.groupBy("state") \
  .sum("salary") \
  .withColumnRenamed("sum(salary)", "sum_salary") \
  .show()

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   NV|    166000|
|   CA|    171000|
|   DE|     99000|
|   NJ|     91000|
+-----+----------+



In [0]:
df.groupBy("state") \
  .sum("salary") \
  .select(col("state"),col("sum(salary)").alias("sum_salary")) \
  .show()
 

+-----+----------+
|state|sum_salary|
+-----+----------+
|   NY|    252000|
|   NV|    166000|
|   CA|    171000|
|   DE|     99000|
|   NJ|     91000|
+-----+----------+

